In [1]:
import IPython
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly as py
from plotly.offline import download_plotlyjs
import pylab
import scipy
import seaborn as sns
import sklearn
from sklearn import *
import statsmodels as sm

np.random.seed(1337)

%matplotlib inline

IPython.core.display.display(IPython.core.display.HTML(
    "<style>.container { width:90% !important; }</style>"))

sns.set(font_scale=1.3)
sns.set_style('whitegrid')
sns.set_palette(sns.color_palette('muted'))

pd.options.display.max_colwidth = 30
pd.options.display.float_format = '{:,.3f}'.format

py.offline.init_notebook_mode(connected=True)

## Load the data

In [2]:
movies = pd.read_csv('data/movies_clean.csv')
ratings = pd.read_csv('data/ratings_train.csv')

## Association rules

In [3]:
min_item_support = 0.01
min_pair_support = 0.001
min_confidence = 0.5

In [4]:
transactions = set()
for _, rating_group in ratings.groupby(['user_id']):
    transactions.add(frozenset(rating_group.movie_id.tolist()))
print(len(transactions))

138493


In [5]:
def find_frequent_items(transactions, min_support):
    item_counter = collections.Counter([item_id for transaction in transactions for item_id in transaction])
    min_count = len(transactions) * min_support
    frequent_items = {}
    for item_id, item_count in item_counter.items():
        if item_count >= min_count:
            frequent_items[item_id] = item_count
    return frequent_items
frequent_movies = find_frequent_items(transactions, min_item_support)
len(frequent_movies)

NameError: name 'collections' is not defined

In [ ]:
def find_frequent_pairs(transactions, frequent_items, min_support):
    frequent_item_ids = set(frequent_items.keys())
    pair_counts = collections.defaultdict(int)
    min_count = len(transactions) * min_support
    frequent_pairs = {}
    i = 1
    for transaction in transactions:
        if i % 1000 == 0:
            if i % 10000 == 0:
                print('{0:0.1f}%'.format(100.0 * i / len(transactions)), end='')
            else:
                print('.', end='')
        i += 1
        frequent_transaction_items = transaction.intersection(frequent_item_ids)
        for pair in itertools.combinations(frequent_transaction_items, 2):
            pair_counts[pair] += 1
    for pair, pair_count in pair_counts.items():
        if pair_count >= min_count:
            frequent_pairs[pair] = pair_count
    return frequent_pairs
frequent_movie_pairs = find_frequent_pairs(transactions, frequent_movies, min_pair_support)
len(frequent_movie_pairs)

In [ ]:
def calculate_association_rules(frequent_items, frequent_pairs, n_transactions):
    rules = []
    for source, source_freq in frequent_items.items():
        for pair, pair_freq in frequent_pairs.items():
            if source in pair:
                target = list(set(pair).difference(set([source])))[0]
                support = 1.0 * pair_freq / n_transactions
                confidence = 1.0 * pair_freq / source_freq
                if confidence > min_confidence:
                    rules.append((source, target, support, confidence))
    return rules
rules = calculate_association_rules(frequent_movies, frequent_movie_pairs, len(transactions))
len(rules)

In [ ]:
rules_dict = []
movie_names_map = dict(zip(movies.movie_id.tolist(), movies.title.tolist()))
for r in rules:
    rules_dict.append({
        'source_id': r[0],
        'source_name': movie_names_map[r[0]],
        'target_id': r[1],
        'target_name': movie_names_map[r[1]],
        'support': r[2],
        'confidence': r[3]})
rules_df = pd.DataFrame(rules_dict)[['source_id', 'source_name', 'target_id', 'target_name', 'support', 'confidence']]
rules_df.info()

In [ ]:
rules_df.sample(20)

In [ ]:
len(rules_df.source_id.unique())

In [ ]:
len(rules_df.target_id.unique())

In [ ]:
rules_df.sort_values(by='support', ascending=False).head(20)

In [ ]:
rules_df.sort_values(by='confidence', ascending=False).head(20)

In [ ]:
rules_df[rules_df.source_id == 52281].sort_values(by=['confidence', 'support'], ascending=False)

In [ ]:
rules_df.to_csv('data/association_rules.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)